In [ ]:
import yaml
import time
import numpy as np
from functools import wraps



class Platform:
    def __init__(self, config_name, table_path):
        self.table = tables # get table...

        with open(config_name, "r") as f:
            config = yaml.load(f, Loader=yaml.FullLoader)

        self.algo_instance = config["plug"]["AlgorithmClass"]
        self.preprocessing_type = config["plug"]["PreprocessingClass"]
        self.run_type = config["plug"]["runType"]
        self.logs = config["plug"]["logs"]

        match self.run_type:
          case 'test':
            self.test()
          case 'compress':
            self.compress()
          case 'decompress':
            self.compress()

    def measure_time(self, log=False):
        def decorator(func):
            def wrapper(*args, **kwargs):
                start_time = time.time()
                result = func(*args, **kwargs)
                end_time = time.time()
                if log:
                    print(f"Function {func.__name__} took {end_time - start_time} seconds to execute.")
                return result
            return wrapper
        return decorator


    @measure_time(log=self.logs)
    def compress(self):
          pass

    def decompress(self):
          pass

    def test(self):
        start_time = time.time()
        compressed_data = algo.compress(np.array(table))
        end_time = time.time()
        compression_rate = len(compressed_data) / len(table)
        decompressed_data = algo.decompression(compressed_data)
        loss_rate = (table - decompressed_data) / table
        result = {
            "algorithm": algo.__name__,
            "table": table,
            "compression_rate": compression_rate,
            "time_taken_to_compress": end_time - start_time,
            "loss_rate": loss_rate
        }
        self.results.append(result)

In [ ]:
print("hello, world")

In [ ]:
import abc

class CompressionAlgorithm(abc.ABC):
    def __init__(self):
        ...
    @property
    @abc.abstractmethod
    def name(self):
        ...

    @abc.abstractmethod
    def compression(self, table):
        ...

    @abc.abstractmethod
    def decompression(self, compressed_data):
        ...

class LearningCompressionAlgorithm(CompressionAlgorithm):

    @abc.abstractmethod
    def fit(self, training_data):
        ...

class NonLearningCompressionAlgorithm(CompressionAlgorithm):
    ...

class LosslessCompressionAlgorithm(CompressionAlgorithm):
    ...

class LossyCompressionAlgorithm(CompressionAlgorithm):
    ...

In [ ]:
class DummyCompressionAlgorithm(LosslessCompressionAlgorithm):
    name = 'Dummy Compression Algorithm'
    def compression(self, table):
        time.sleep(2)
        return table
    def decompression(self, compressed_data):
        time.sleep(1)
        return compressed_data

In [ ]:
class Processing(abc.ABC):
    def __init__(self):
        ...
    @property
    @abc.abstractmethod
    def name(self):
        ...

    @abc.abstractmethod
    def preprocess(self, raw_table):
        ...

    @abc.abstractmethod
    def postprocess(self, processed_table):
        ...

In [ ]:
class DummyProcessing(Processing):
    name = "Dummy Pre/Post-Processing"
    def preprocess(self, raw_table):
        return raw_table
    def postprocess(self, processed_table):
        return processed_table


In [ ]:
import numpy as np
platform = Platform([DummyCompressionAlgorithm()], [np.array([1,2,3])])
platform.compress_table()
platform.results